# Script Principal para Web Scraping



In [ ]:
import io
import os
import time

import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException

from process_data import process_table, save_data, check_process, update_register

def run_scraper(years, regions=None, download_folder='output', check_processed=True,
               headless=False, shutdown_on_complete=False):

    if not os.path.exists(download_folder):
        os.makedirs(download_folder)

    chrome_options = Options()
    # Configurar preferencias de descarga
    prefs = {
        "download.default_directory": os.path.abspath(download_folder),
        "download.prompt_for_download": False,
        "download.directory_upgrade": True,
        "safebrowsing.enabled": True,
        "plugins.always_open_pdf_externally": True,
        "download.extensions_to_open": "",
        "profile.default_content_settings.popups": 0,
        "profile.default_content_setting_values.automatic_downloads": 1,
        "safebrowsing.disable_download_protection": True
    }
    # Configuraciones para estabilidad y rendimiento

    chrome_options.add_experimental_option("prefs", prefs)
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    if headless:
        chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--disable-popup-blocking")
    chrome_options.add_argument("--disable-extensions")
    chrome_options.add_argument("--disable-notifications")
    chrome_options.add_argument("--disable-infobars")
    chrome_options.add_argument("--ignore-certificate-errors")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")
    chrome_options.add_argument("--disable-features=NetworkService")
    chrome_options.add_argument("--dns-prefetch-disable")
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-setuid-sandbox")

    try:
        URL = "http://sistemas.midagri.gob.pe/sisap/portal2/ciudades/"

        driver = webdriver.Chrome(options=chrome_options)
        driver.set_window_position(x=-1930, y=-1)
        driver.maximize_window()
        driver.get(URL)
        driver.refresh()

        wait = WebDriverWait(driver, 8)

        complete_wait_condition = lambda d: d.execute_script("return document.readyState") == "complete"
        wait.until(complete_wait_condition)

        # Obtención de regiones para automatización
        region_we = driver.find_element(By.ID, 'region')

        # Creamos diccionario de regiones (nombre: valor) para facilitar selección
        all_regions = {}
        for option in region_we.find_elements(By.TAG_NAME, 'option'):
            all_regions[option.text] = option.get_attribute('value')

        # Filtrar regiones
        selected_regions = {k: v for k, v in all_regions.items() if k != 'Seleccionar Región...'}
        # Descartamos que ingresen nombres de provincia mal escritos o que no existan
        if regions is not None:
            invalid = [r for r in regions if r not in selected_regions]
            if invalid:
                print(f"Regiones inválidas ignoradas: {invalid}")
            selected_regions = {k: v for k, v in selected_regions.items() if k in regions}


        for year in years:

            #  Iteramos en regiones
            for region_name, region_value in selected_regions.items():

                # Escoger Región
                time.sleep(.5)
                Select(region_we).select_by_value(region_value)
                print(f'Procesando {region_name} - {year}')
                time.sleep(1.5)

                ###################################### VARIABLES ##############################
                variables_container_we = driver.find_element(By.ID, 'variables')
                variables_may_container_we = variables_container_we.find_elements(By.XPATH, './optgroup[1]/option')
                variables_min_container_we = variables_container_we.find_elements(By.XPATH, './optgroup[2]/option')

                # Hacer clic en las 6 variables
                [x.click() for x in variables_may_container_we + variables_min_container_we]


                ###################################### INTERVALO DE FECHAS #####################
                # intervalo de fechas
                driver.find_element(By.XPATH, '//*[@id="intervalo"]').click()
                time.sleep(.3)

                ###################################### CALENDARIO DESDE ########################
                # Click en input DESDE
                driver.find_element(By.XPATH, '//*[@id="desde"]').click()
                time.sleep(.2)
                # click en select año
                select_year_desde = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[2]')
                Select(select_year_desde).select_by_value(str(year))
                time.sleep(.3)
                # click en mes de inicio de año
                select_mes_desde = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[1]')
                Select(select_mes_desde).select_by_value('0')
                time.sleep(.4)
                # click en dia de inicio de mes
                tbody = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody')
                days = tbody.find_elements(By.TAG_NAME, 'a')
                # indicar 0 para indicar el primer dia del mes
                driver.execute_script("arguments[0].click();", days[0])
                # days[0].click()
                time.sleep(.2)

                ####################################### CALENDARIO HASTA ###########################
                # Click en input HASTA
                driver.find_element(By.XPATH, '//*[@id="hasta"]').click()
                time.sleep(.2)

                # Click en select año
                select_year_hasta = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[2]')
                Select(select_year_hasta).select_by_value(str(year))
                time.sleep(.3)
                # click en el mes final del año
                select_mes_hasta = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/div/div/select[1]')
                Select(select_mes_hasta).select_by_value('11')
                time.sleep(.4)
                # click en el dia final del mes
                tbody = driver.find_element(By.XPATH, '//*[@id="ui-datepicker-div"]/table/tbody')
                days = tbody.find_elements(By.TAG_NAME, 'a')
                # indicar -1 para indicar el ultimo dia del mes
                days[-1].click()
                time.sleep(.3)
                ############################## SELECCIONAR PRODUCTO Y SUB-PRODUCTO ####################
                # TOMA DE DATOS
                # Obtención de productos para automatización
                products_container = driver.find_element(By.ID, 'productosCheckBox')

                # Creamos diccionario de regiones y botones expansores (nombre_producto: boton_web_element) para facilitar selección
                all_products = {}
                for option in products_container.find_elements(By.XPATH, './li'):
                    all_products[option.find_element(By.XPATH, './label').text] = option.find_element(By.XPATH, './a')

                # Modifica el estilo height a max-content usando JavaScript
                box_products = driver.find_element(By.ID, 'productos')
                driver.execute_script("arguments[0].style.height = 'max-content';", box_products)


                # Modifica el estilo height a max-content usando JavaScript
                box_products = driver.find_element(By.ID, 'productos')
                driver.execute_script("arguments[0].style.height = 'max-content';", box_products)

                def click_with_js(elements):
                    for element in elements:
                        driver.execute_script("arguments[0].click();", element)

                # expandir los sub-productos para que puedan ser leidos por selenium
                click_with_js(all_products.values())

                # Obtención de sub-productos para automatización
                products_container = driver.find_element(By.ID, 'productosCheckBox')

                # Creamos diccionario de regiones y sub-productos (nombre_producto: { nombre_sub-producto : sub_producto_web_element })
                all_sub_products = {}
                for product in products_container.find_elements(By.XPATH, './li'):
                    sub_products = {}
                    for sub_p in product.find_elements(By.XPATH, './ul/li'):
                        sub_products[sub_p.find_element(By.XPATH, './label').text] = sub_p.find_element(By.XPATH, './label')
                    all_sub_products[product.find_element(By.XPATH, './label').text] = sub_products

                # Comprimir los sub-productos
                click_with_js(all_products.values())

                # Modifica el estilo height a 300px usando JavaScript
                driver.execute_script("arguments[0].style.height = '300px';", box_products)

                # ITERACION EN PRODUCTOS Y SUBPRODUCTOS
                for product_name, product_we in all_products.items():

                    # if product_name not in ['Papa']:
                    #     continue
                    # expandir la lista de sub-producto
                    driver.execute_script("arguments[0].click();", product_we)

                    for sub_product_name, sub_product_we in all_sub_products[product_name].items():

                        if check_processed and check_process(year, region_name, product_name, sub_product_name, download_folder):
                            print(f"Saltando {year} - {region_name} - {product_name} - {sub_product_name} (ya procesado)")
                            continue
                        # Marcar sub-producto
                        driver.execute_script("arguments[0].click();", sub_product_we)
                        print(f'click in {sub_product_name}')

                        ########################################### CONSULTAR #################################

                        consult = driver.find_element(By.ID, 'consultar')
                        driver.execute_script("arguments[0].click();", consult)
                        ############################################ DATOS ####################################

                        try:
                            table = (By.XPATH, '//*[@id="reporte"]/table')
                            message_error = (By.CLASS_NAME, 'mensajeDeError')

                            result = wait.until(EC.any_of(
                                EC.presence_of_element_located(table),
                                EC.presence_of_element_located(message_error)
                            ))
                            if result.tag_name == 'table':
                                table_html = result.get_attribute('outerHTML')
                                df = pd.read_html(io.StringIO(table_html))[0]
                            else:
                                error_msg = result.text
                                truncated_msg = (error_msg[:47] + '...') if len(error_msg) > 50 else error_msg
                                print(f"No se encontraron datos en {sub_product_name}: {truncated_msg}")
                                driver.execute_script('arguments[0].click()', sub_product_we)
                                # Guardar registro
                                update_register(year, region_name, product_name, sub_product_name, download_folder)

                                continue
                        except TimeoutException:
                            print(f"Error de tiempo de espera en {sub_product_name}")
                            driver.execute_script('arguments[0].click()', sub_product_we)
                            continue

                        except Exception as err:
                            print(f"Error inesperado: {err}")
                            driver.execute_script('arguments[0].click()', sub_product_we)
                            continue

                        time.sleep(.2)

                        # Procesar tabla
                        df_mayoristas, df_minoristas = process_table(df, region_name, product_name, sub_product_name, year)

                        # Guardar registro
                        save_data(df_mayoristas, df_minoristas, year, region_name, product_name, sub_product_name, download_folder )

                        # Desmarcar sub-producto
                        driver.execute_script("arguments[0].click();", sub_product_we)

        # Después de completar todo el proceso exitosamente
        if shutdown_on_complete:
            print("\nProceso completado. Apagando el sistema en 10 segundos...")
            shutdown_command = "shutdown /s /t 10" if os.name == 'nt' else "shutdown -h now"
            os.system(shutdown_command)

    except Exception as e:
        print(f"\nError durante el proceso: {str(e)}")
        print("El apagado automático fue cancelado debido al error")

    finally:
        driver.quit()
        print("\nControlador del navegador cerrado correctamente")



In [ ]:
# Lista de años disponibles para scraping (2015-2025)
years = [2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025]

# Ejemplo de uso con explicación de cada parámetro:
run_scraper(
    # Lista de años a scrapear (obligatorio)
    # Ej: [2023] para un solo año o [2022,2023,2024,etc] para múltiples años
    years=[2023, 2024],

    # Lista de regiones a scrapear (opcional)
    # None: procesa todas las regiones disponibles
    # Ej: ['Lima'] para una región o ['Lima', 'Arequipa',etc] para múltiples
    regions=['Lima'],

    # Carpeta donde se guardarán los archivos descargados (opcional)
    # Default: 'output' (se creará automáticamente si no existe)
    download_folder='output',

    # Verificación de archivos ya procesados (opcional)
    # True: omite productos/subproductos ya scrapeados (ahorra tiempo)
    # False: reprocesa todo (útil para actualizar datos)
    check_processed=True,

    # Ejecución sin interfaz gráfica (opcional)
    # True: modo headless (sin mostrar navegador)
    # False: muestra el navegador durante el scraping
    headless=False,

    # Apagado automático de la pc al completar (opcional)
    # True: apaga el equipo tras finalizar exitosamente
    # False: no realiza acción al finalizar (recomendado para pruebas)
    shutdown_on_complete=False
)

# Web Scraping de Precios

Enlace: http://sistemas.midagri.gob.pe/sisap/portal/index.php

Se necesita la siguiente información:
- regiones: Junin, Huancavelica, Apurimac, Ayacucho
- productos:  Papa, Quinoa, Habas, Trigo, Alfalfa, Cebada, Maíz, Oca, Mashua, Olluco
- dias: antes de 14 de marzo de 2025 hasta 1 enero 2025
- meses: febrero 2025 hasta enero 2024
- años: 2025-2015
